# LYN

Data from [Louvrier C, et al. (2023) De Novo Gain-Of-Function Variations in LYN Associated With an Early-Onset Systemic Autoinflammatory Disorder. Arthritis Rheumatol;75(3):468-474. PMID:36122175](https://pubmed.ncbi.nlm.nih.gov/36122175/).

In [11]:
import pandas as pd
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
from pyphetools.creation import *
from pyphetools.visualization import PhenopacketTable, QcVisualizer
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.20


In [12]:
PMID="PMID:36122175"
title = "De Novo Gain-Of-Function Variations in LYN Associated With an Early-Onset Systemic Autoinflammatory Disorder"
cite = Citation(pmid=PMID, title=title)
parser = HpoParser(hpo_json_file="../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
metadata = MetaData(created_by="ORCID:0000-0002-0736-9199", citation=cite)
metadata.default_versions_with_hpo(version=hpo_version)
print(f"HPO version {hpo_version}")

HPO version 2023-10-09


In [13]:
df = pd.read_excel("input/louvrier_2023_LYN.xlsx")
dft = df.transpose()
dft.columns = dft.iloc[0]
dft.drop(dft.index[0], inplace=True)
dft['patient_id'] = dft.index  # Set the new column 'patient_id' to be identical to the contents of the index
dft.head() # check the transposed table

Clinical and laboratory features,Age of disease onset,Age at diagnosis,Presenting Symptoms,Hydrops fetalis,Liver fibrosis,Other clinical manifestations,ESR (mm/1h),CRP (mg/L),SAA (mg/L),CBC,LFTs,Autoantibodies,Complement,CD4/CD8/B/NK lymphocytes (abs #),IgG/IgA/IgM,Skin biopsies,Elastography (max),Infections,Treatment,patient_id
Patient 1,1st day of life,2 years 6 months old,"Purpuric rash, hepatosplenomegaly, fever, thrombocytopenia at birth","Yes, intra-utero platelet and PRBC transfusion at 29 weeks of GA",Yes,"Recurrent parotitis, abdominal pain, periorbital edema and erythema, conjunctivitis, epididymitis, headaches, arthralgias, oral ulcers, fatigue, GVHD-like colitis",64,14–86.5,ND,"Mild anemia, mild leukocytosis, moderate to severe thrombocytopenia","Increased ALT, AST and GGT","Positive ANA, anti-Sm, anti-SSA, ACL IgG, LAC, anti-mitochondrial, RF, anti-TPO","nl CH50, C3, C4",High/high/nl/nl,nl/nl/low,Small vessel vasculitis with neutrophilic infiltrate and destruction of dermal vessel walls,6.7 kPa,"Enteropathogenic E. coli, Salmonella sp, Toxocara canisb","Poor response to IVIG, IVMP and oral prednisolone, partial response to dasatinib monotherapy, partial response to etanercept monotherapy, good response to dasatinib and etanercept combination therapy",Patient 1
Patient 2,1st day of life,15 years-old,"Mild purpuric rash at birth, fever, and generalized severe purpuric rash at the age of 3 months",No,No,"Recurrent abdominal pain, periorbital edema and erythema, conjunctivitis, epididymitis, headaches, arthralgias, oral ulcers, fatigue, GVHD-like colitis",ND,46–166,182–984,Mild leukocytosis,"Normal ALT, AST and GGT","Negative by clinical tests, transient positivity for ANA on research testing once","nl C3, C4",nl/nl/nl/nl,nl/nl/low,Perivascular neutrophilic dermal infiltrate,"ND, normal LFTs",Post-streptococcal glomerulonephritis,"No response to anakinra and tocilizumab, partial response to colchicine and good response to etanercept and colchicine",Patient 2
Patient 3,1st day of life,4 months old,"Hepatosplenomegaly, thrombocytopenia, and discrete purpuric rash at birth","No, had congenital hydrocele",Yes,"Intrauterine growth restriction, failure to thrive, transient periorbital erythema, jaundice, direct hyperbilirubinemia",ND,6.5–107.6,ND,"Mild anemia, moderate leukocytosis, moderate to severe thrombocytopenia","Increased ALT, AST and GGT",Borderline anti-PR3,ND,High/high/high/high,nl/low/nl,Small vessel vasculitis with neutrophilic infiltrate and destruction of dermal vessel walls,18.7 kPa,"Late neonatal sepsis, COVID-19 (asymptomatic)","Partial improvement of CRP and thrombocytopenia, resolution of direct hyperbilirubinemia, return of normal growth rate, and persistence of liver fibrosis on etanercept therapy",Patient 3


In [15]:
res = OptionColumnMapper.autoformat(df=dft, concept_recognizer=hpo_cr, )

NameError: name 'df_name' is not defined